In [16]:
data_rep = "../data/"

from conllu import parse_incr
from conllulib import CoNLLUReader

# Read the CoNLL-U file and parse it into sentenceader.to_bio(sentences)
test = CoNLLUReader.readConllu(data_rep+"sequoia-ud.parseme.frsemcor.simple.small")

CoNLLUReader.to_bio(test)

# # Print the parsed sentences
# print(sentences)

# def transform_to_bio(sentences):
#     bio_list = []
#     for sent in sentences:
#         for tok in sent:
#             print(tok)
#             bio_list.append((tok["form"], tok["parseme:ne"]))
#     return bio_list

# # Transform the parsed sentences to BIO format
# bio_data = transform_to_bio(sentences)
# print(bio_data)

AttributeError: 'str' object has no attribute 'infile'

In [6]:
bio_list = []
for sent in parse_incr(open(data_rep + "sequoia-ud.parseme.frsemcor.simple.small", encoding='UTF-8')):
    for tok in sent:
        if tok["parseme:ne"] == '*':
            bio_list.append((tok["form"], 'O'))
        else:
            ne_parts = tok["parseme:ne"].split(':')
            if len(ne_parts) > 1:
                ne_type = ne_parts[1]
                if ne_parts[0] == '1':
                    bio_list.append((tok["form"], 'B-' + ne_type))
                else:
                    bio_list.append((tok["form"], 'I-' + ne_type))
            else:
                bio_list.append((tok["form"], 'O'))  # Change 'B-' to 'O' to avoid standalone 'B-'
print(bio_list)

[("L'", 'O'), ('association', 'O'), ('a', 'O'), ('changé', 'O'), ('les', 'O'), ('décors', 'O'), ('et', 'O'), ('avec', 'O'), ("l'", 'O'), ('aide', 'O'), ('de', 'O'), ('plusieurs', 'O'), ('bénévoles', 'O'), (',', 'O'), ('établi', 'O'), ('différents', 'O'), ('tableaux', 'O'), ('sur', 'O'), ('le', 'O'), ('thème', 'O'), ('de', 'O'), ('"', 'O'), ('Cinq', 'O'), ('siècles', 'O'), ("d'", 'O'), ('activité', 'O'), ('économique', 'O'), ('de', 'O'), ('la', 'O'), ('région', 'O'), ("d'", 'O'), ('Ancerville', 'B-LOC'), ('"', 'O'), ('.', 'O'), ('Ouverture', 'O'), ('tous', 'O'), ('les', 'O'), ('jours', 'O'), ('sauf', 'O'), ('le', 'O'), ('lundi', 'O'), ('de', 'O'), ('14', 'O'), ('h', 'O'), ('30', 'O'), ('à', 'O'), ('18', 'O'), ('h', 'O'), ('.', 'O'), ('Quant', 'O'), ('à', 'O'), ('le', 'O'), ('sous-préfet', 'O'), (',', 'O'), ('il', 'O'), ('apprécie', 'O'), ("l'", 'O'), ('énergie', 'O'), ('dépensée', 'O'), ('pour', 'O'), ('une', 'O'), ('telle', 'O'), ('réalisation', 'O'), ('.', 'O'), ('Les', 'O'), ('membre

In [12]:
import math
import pickle

def extraction(bio_list):
    c_wj_ti = {}
    c_ti_tj = {}
    c_ti = {}
    c_s_ti = {}

    prev_tag = None

    for i, (word, tag) in enumerate(bio_list):
        # Count occurrences of each word-tag pair
        if (word, tag) not in c_wj_ti:
            c_wj_ti[(word, tag)] = 0
        c_wj_ti[(word, tag)] += 1

        # Count occurrences of each tag
        if tag not in c_ti:
            c_ti[tag] = 0
        c_ti[tag] += 1

        # Count occurrences of each adjacent tag pair
        if prev_tag is not None:
            if (prev_tag, tag) not in c_ti_tj:
                c_ti_tj[(prev_tag, tag)] = 0
            c_ti_tj[(prev_tag, tag)] += 1

        # Count occurrences of each tag at the beginning of a sentence
        if i == 0 or bio_list[i-1][1] == 'O':
            if ('<s>', tag) not in c_s_ti:
                c_s_ti[('<s>', tag)] = 0
            c_s_ti[('<s>', tag)] += 1

        prev_tag = tag

    return c_wj_ti, c_ti_tj, c_ti, c_s_ti

c_wj_ti, c_ti_tj, c_ti, c_s_ti = extraction(bio_list)
print(c_wj_ti)
print(c_ti_tj)
print(c_ti)
print(c_s_ti)



log_prob_wj_ti = {}
log_prob_ti_tj = {}
log_prob_pi_ti = {}

# Calculate log probabilities for word-tag pairs
for (word, tag), count in c_wj_ti.items():
    log_prob_wj_ti[(word, tag)] = math.log(c_ti[tag]) - math.log(count)

# Calculate log probabilities for tag-tag transitions
for (prev_tag, tag), count in c_ti_tj.items():
    log_prob_ti_tj[(prev_tag, tag)] = math.log(c_ti[prev_tag]) - math.log(count)

# Calculate log probabilities for tags at the beginning of a sentence
S = sum(c_s_ti.values())
for (start, tag), count in c_s_ti.items():
    log_prob_pi_ti[tag] = math.log(S) - math.log(count)

print(log_prob_wj_ti)
print(log_prob_ti_tj)
print(log_prob_pi_ti)


with open('dicts.pkl', 'wb') as f:
    pickle.dump({'c_wj_ti': c_wj_ti, 'c_ti_tj': c_ti_tj, 'c_ti': c_ti, 'c_s_ti': c_s_ti, 'log_prob_wj_ti': log_prob_wj_ti, 'log_prob_ti_tj': log_prob_ti_tj, 'log_prob_pi_ti': log_prob_pi_ti}, f)

{("L'", 'O'): 4, ('association', 'O'): 2, ('a', 'O'): 2, ('changé', 'O'): 1, ('les', 'O'): 20, ('décors', 'O'): 1, ('et', 'O'): 13, ('avec', 'O'): 3, ("l'", 'O'): 13, ('aide', 'O'): 1, ('de', 'O'): 46, ('plusieurs', 'O'): 2, ('bénévoles', 'O'): 1, (',', 'O'): 35, ('établi', 'O'): 1, ('différents', 'O'): 1, ('tableaux', 'O'): 1, ('sur', 'O'): 1, ('le', 'O'): 30, ('thème', 'O'): 1, ('"', 'O'): 15, ('Cinq', 'O'): 1, ('siècles', 'O'): 1, ("d'", 'O'): 16, ('activité', 'O'): 2, ('économique', 'O'): 1, ('la', 'O'): 9, ('région', 'O'): 1, ('Ancerville', 'B-LOC'): 1, ('.', 'O'): 26, ('Ouverture', 'O'): 1, ('tous', 'O'): 1, ('jours', 'O'): 2, ('sauf', 'O'): 1, ('lundi', 'O'): 1, ('14', 'O'): 3, ('h', 'O'): 3, ('30', 'O'): 1, ('à', 'O'): 17, ('18', 'O'): 1, ('Quant', 'O'): 1, ('sous-préfet', 'O'): 1, ('il', 'O'): 5, ('apprécie', 'O'): 1, ('énergie', 'O'): 1, ('dépensée', 'O'): 1, ('pour', 'O'): 6, ('une', 'O'): 4, ('telle', 'O'): 1, ('réalisation', 'O'): 1, ('Les', 'O'): 2, ('membres', 'O'): 1, (

In [ ]:
import numpy as np
from conllulib import from_bio

with open('dicts.pkl', 'rb') as f:
    b = pickle.load(f)


    # Define constants
    N = len(c_ti)  # Number of unique tags
    PSEUDOINF = float('inf')

    # Initialize delta and psi tables
    delta = np.full((N, len(bio_list)), PSEUDOINF)
    psi = np.zeros((N, len(bio_list)), dtype=int)

    # Create tag to index mapping
    tag_to_index = {tag: idx for idx, tag in enumerate(c_ti.keys())}
    index_to_tag = {idx: tag for tag, idx in tag_to_index.items()}

    # Initialization (column 1)
    for j, tag in enumerate(c_ti.keys()):
        word = bio_list[0][0]
        delta[j, 0] = -log_prob_pi_ti.get(tag, PSEUDOINF) - log_prob_wj_ti.get((word, tag), PSEUDOINF)

    # Recursive step
    for k in range(1, len(bio_list)):
        word = bio_list[k][0]
        for j, tag in enumerate(c_ti.keys()):
            min_val = PSEUDOINF
            min_index = 0
            for i, prev_tag in enumerate(c_ti.keys()):
                val = delta[i, k-1] - log_prob_ti_tj.get((prev_tag, tag), PSEUDOINF)
                if val < min_val:
                    min_val = val
                    min_index = i
            delta[j, k] = min_val - log_prob_wj_ti.get((word, tag), PSEUDOINF)
            psi[j, k] = min_index

    # Backtracking to find the optimal tag sequence
    n = len(bio_list)
    optimal_tags = [None] * n
    optimal_tags[-1] = index_to_tag[np.argmin(delta[:, -1])]

    for k in range(n-2, -1, -1):
        optimal_tags[k] = index_to_tag[psi[tag_to_index[optimal_tags[k+1]], k+1]]

    # Convert BIO tags back to parseme format
    predicted_sentences = from_bio([(bio_list[i][0], optimal_tags[i]) for i in range(n)])
    print(predicted_sentences)